```
★프로토 타입 개발: 간단한 식단 추천 시스템을 먼저 구현, 음식 DB와 기본 추천 알고리즘을 사용해 프로토 타입 개발

아이디어 :1. 개인 영양상태나 질병에 따라 영양별 위험군 입력시 그에 해당하는 값이 계산되어 그에 해당되는 식단을 표로 작성
2. 식재료 가격 변동 추이에 따라 최적의 가격으로 식단을 작성(RDA을 기준으로 배분)+ 
프로젝트 목적: 

프로젝트 목표설정:
데이터 수집:
모델설계:


```

In [28]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:90%;}
div.CodeMirror {font-family:Consolas; font-size:13pt;}
div.output {font-size:13pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:13pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:13pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
table.dataframe{font-size:13pt;}
</style>
"""))

In [1]:
import pandas as pd

# 엑셀 파일 읽기

eat_origin_code = df = pd.read_excel('../data/급식식재료표준코드.xlsx',dtype=str)
eat_origin_code.head()

C:\ai\IDE\anaconda\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,대분류코드,대분류명,중분류코드,중분류명,소분류코드,소분류명,공통코드(4자리),식품명,비고
0,1,농산물,01,견과류,001,개암,0001,개암,NaN
1,1,농산물,01,견과류,001,개암,0002,개암/말린것(건조),NaN
2,1,농산물,01,견과류,001,개암,0003,개암/볶은것,NaN
3,1,농산물,01,견과류,002,도토리,0001,도토리,NaN
4,1,농산물,01,견과류,002,도토리,0002,도토리/국산,NaN


In [3]:
eat_origin_code

,대분류코드,대분류명,중분류코드,중분류명,소분류코드,소분류명,공통코드(4자리),식품명
0,1,농산물,01,견과류,001,개암,0001,개암
1,1,농산물,01,견과류,001,개암,0002,개암/말린것(건조)
2,1,농산물,01,견과류,001,개암,0003,개암/볶은것
3,1,농산물,01,견과류,002,도토리,0001,도토리
4,1,농산물,01,견과류,002,도토리,0002,도토리/국산
...,...,...,...,...,...,...,...,...
9515,4,가공식품,29,후식류,047,호떡(뽕잎호떡),0001,호떡(뽕잎호떡)
9516,4,가공식품,29,후식류,048,호떡(쌀호떡),0001,호떡(쌀호떡)
9517,4,가공식품,29,후식류,049,호떡(찹쌀호떡),0001,호떡(찹쌀호떡)
9518,4,가공식품,29,후식류,050,호떡(호두호떡),0001,호떡(호두호떡)


In [4]:
df =eat_origin_code[['대분류코드','중분류코드','소분류코드','공통코드(4자리)']]
eat_origin_code['식재료코드']  = df.apply(lambda row: ''.join(row), axis=1)
eat_origin_code.drop('대분류코드',axis=1,inplace=True)
eat_origin_code.drop('중분류코드',axis=1,inplace=True)
eat_origin_code.drop('소분류코드',axis=1,inplace=True)
eat_origin_code.drop('공통코드(4자리)',axis=1,inplace=True)
eat_origin_code

,대분류명,중분류명,소분류명,식품명,식재료코드
0,농산물,견과류,개암,개암,1010010001
1,농산물,견과류,개암,개암/말린것(건조),1010010002
2,농산물,견과류,개암,개암/볶은것,1010010003
3,농산물,견과류,도토리,도토리,1010020001
4,농산물,견과류,도토리,도토리/국산,1010020002
...,...,...,...,...,...
9515,가공식품,후식류,호떡(뽕잎호떡),호떡(뽕잎호떡),4290470001
9516,가공식품,후식류,호떡(쌀호떡),호떡(쌀호떡),4290480001
9517,가공식품,후식류,호떡(찹쌀호떡),호떡(찹쌀호떡),4290490001
9518,가공식품,후식류,호떡(호두호떡),호떡(호두호떡),4290500001


In [7]:
result = []
for food in eat_origin_code['식품명'].tolist():
    result.append(food)


print(food)


호떡(흑미호떡)


In [18]:
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote

# 인증키
key = 'Hy7rvWaXikWqGQCbQxWU9Hyzj5AXoFgds%2ByZ1knY9F%2FfEON853OPBRo8OYpKYvCiEB3r8uYZ1ZpU4EMpppX%2B8g%3D%3D'

# 결과 리스트 초기화
nutrition_info = []

# 총 데이터 개수
total = len(eat_origin_code['식품명'])

# 시간 측정 시작
start_time = time.time()

# 데이터 처리 및 API 호출
for idx, food in enumerate(eat_origin_code['식품명'].tolist(), start=1):
    # 진행 상태 출력 (1% 단위)
    if idx % (total // 100) == 0:  # 전체의 1%마다
        progress = (idx / total) * 100
        print(f"진행률: {progress:.0f}% 완료")

    # API 호출 URL 생성
    url = f'https://apis.data.go.kr/1471000/FoodNtrCpntDbInfo01/getFoodNtrCpntDbInq01?serviceKey={key}&numOfRows=100&type=xml&FOOD_NM_KR={quote(food)}'
    response = requests.get(url)
    
    if response.status_code == 200:  # API 호출 성공
        soup = BeautifulSoup(response.text, "xml")
        itemLists = soup.find_all('item')
        
        for itemList in itemLists:
            try:
                FOOD_NM_KR = itemList.find('FOOD_NM_KR').text
                AMT_NUM1 = itemList.find('AMT_NUM1').text
                AMT_NUM7 = itemList.find('AMT_NUM7').text
                AMT_NUM3 = itemList.find('AMT_NUM3').text
                AMT_NUM4 = itemList.find('AMT_NUM4').text
                NUTRI_AMOUNT_SERVING=itemList.find('NUTRI_AMOUNT_SERVING').text
                Z10500 = itemList.find('Z10500').text

                nutrition_info.append([FOOD_NM_KR, AMT_NUM1, AMT_NUM7, AMT_NUM3, AMT_NUM4,NUTRI_AMOUNT_SERVING,Z10500])
            except AttributeError:
                # 일부 데이터가 없을 경우 대비
                continue
    else:
        print(f"Failed to fetch data for {food}. HTTP Status Code: {response.status_code}")

# 시간 측정 종료
end_time = time.time()

# 최종 완료 메시지 및 소요 시간 출력
print("처리 완료")
print(f"총 소요 시간: {end_time - start_time:.6f} 초")

# 데이터프레임 생성
df = pd.DataFrame(nutrition_info, columns=['식재료명', '에너지(kcal)', '탄수화물', '단백질', '지방','1회 섭취 참고량','식품중량'])

# 결과 출력
df


진행률: 1% 완료
진행률: 2% 완료
진행률: 3% 완료
진행률: 4% 완료
진행률: 5% 완료
진행률: 6% 완료
진행률: 7% 완료
진행률: 8% 완료
진행률: 9% 완료
진행률: 10% 완료
진행률: 11% 완료
진행률: 12% 완료
진행률: 13% 완료
진행률: 14% 완료
진행률: 15% 완료
진행률: 16% 완료
진행률: 17% 완료
진행률: 18% 완료
진행률: 19% 완료
진행률: 20% 완료
진행률: 21% 완료
진행률: 22% 완료
진행률: 23% 완료
진행률: 24% 완료
진행률: 25% 완료
진행률: 26% 완료
진행률: 27% 완료
진행률: 28% 완료
진행률: 29% 완료
진행률: 30% 완료
진행률: 31% 완료
진행률: 32% 완료
진행률: 33% 완료
진행률: 34% 완료
진행률: 35% 완료
진행률: 36% 완료
진행률: 37% 완료
진행률: 38% 완료
진행률: 39% 완료
진행률: 40% 완료
진행률: 41% 완료
진행률: 42% 완료
진행률: 43% 완료
진행률: 44% 완료
진행률: 45% 완료
진행률: 46% 완료
진행률: 47% 완료
진행률: 48% 완료
진행률: 49% 완료
진행률: 50% 완료
진행률: 51% 완료
진행률: 52% 완료
진행률: 53% 완료
진행률: 54% 완료
진행률: 55% 완료
진행률: 56% 완료
진행률: 57% 완료
진행률: 58% 완료
진행률: 59% 완료
진행률: 60% 완료
진행률: 61% 완료
진행률: 62% 완료
진행률: 63% 완료
진행률: 64% 완료
진행률: 65% 완료
진행률: 66% 완료
진행률: 67% 완료
진행률: 68% 완료
진행률: 69% 완료
진행률: 70% 완료
진행률: 71% 완료
진행률: 72% 완료
진행률: 73% 완료
진행률: 74% 완료
진행률: 75% 완료
진행률: 76% 완료
진행률: 77% 완료
진행률: 78% 완료
진행률: 79% 완료
진행률: 80% 완료
진행률: 81% 완료
진행률: 82% 완료
진행률: 83% 완료
진행률: 84% 완료
진

,식재료명,에너지(kcal),탄수화물,단백질,지방,1회 섭취 참고량,식품중량
0,개암(헤이즐넛)_말린것,628.000,16.70,14.95,60.75,,
1,개암(헤이즐넛)_볶은것,646.000,17.60,15.03,62.40,,
2,묵무침_도토리묵,74.00,10.92,0.93,2.95,,200.000g
3,묵말이_도토리묵,36.00,6.90,0.52,0.65,,"1,300.000g"
4,묵무침_도토리묵,46.00,10.38,0.29,0.32,,100.000g
...,...,...,...,...,...,...,...
31199,굿프랜즈 우리동네 꿀호떡,342.000,57.00,4.00,11.00,70g,100.000g
31200,단백질꿀호떡,267.000,52.00,8.67,2.53,70g,150.000g
31201,달달 찹쌀호떡,270.000,55.00,5.00,5.00,70g,"1,600.000g"
31202,더벤티 씨앗호떡,359.000,53.00,7.00,13.00,70g,100.000g


In [ ]:
# api 사용후 결과물 csv파일로 저장

df.to_csv(r'../data/식재료별영양소함량.csv',index=False,encoding='utf-8')


In [29]:
df = pd.read_csv(r'../data/식재료별영양소함량.csv',encoding='utf-8')
df

,식재료명,에너지(kcal),탄수화물,단백질,지방,1회 섭취 참고량,식품중량
0,개암(헤이즐넛)_말린것,628.000,16.70,14.95,60.75,NaN,NaN
1,개암(헤이즐넛)_볶은것,646.000,17.60,15.03,62.40,NaN,NaN
2,묵무침_도토리묵,74.00,10.92,0.93,2.95,NaN,200.000g
3,묵말이_도토리묵,36.00,6.90,0.52,0.65,NaN,"1,300.000g"
4,묵무침_도토리묵,46.00,10.38,0.29,0.32,NaN,100.000g
...,...,...,...,...,...,...,...
31199,굿프랜즈 우리동네 꿀호떡,342.000,57.00,4.00,11.00,70g,100.000g
31200,단백질꿀호떡,267.000,52.00,8.67,2.53,70g,150.000g
31201,달달 찹쌀호떡,270.000,55.00,5.00,5.00,70g,"1,600.000g"
31202,더벤티 씨앗호떡,359.000,53.00,7.00,13.00,70g,100.000g


In [30]:
df.loc['도토리묵']

KeyError: '도토리묵'